## Federated data integration testing

In [3]:
import sys
import os
sys.path.append(".../RADAR_Plataform/RADAR") # dir library


##### Example of static data set for testing

In [4]:
from RADAR.static_data.static_datasets_uci import global_load
from sklearn.model_selection import train_test_split
X,y = global_load('default_of_credit_card_clients')   #name dataset in static datasets uci repo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

2025-10-14 13:43:13.139283: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 13:43:13.151379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760442193.165579   40807 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760442193.169358   40807 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-14 13:43:13.183884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Metadata: {'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 365, 'type': 'NATIVE', 'title': 'The comparisons of data mining techniques for the predictive ac

#### Tests with different models of the flex-anomaly platform

##### Iforest

In [5]:
from RADAR.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "isolationForest",
    "contamination":0.1,
    "label_parser": None,
    "n_estimators": 100, 
    "n_rounds": 10,
    "n_clients":5,
    
}

modelIforest = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelIforest.get_params())

Federated Params:{'n_rounds': 10, 'n_clients': 5} 
 Model Params:{'algorithm_': 'isolationForest', 'contamination': 0.1, 'label_parser': None, 'n_estimators': 100}
Inspecting model's attributes:
contamination: 0.1
n_estimators: 100
max_samples: 1000
max_features: 1.0
bootstrap: False
n_jobs: 1
behaviour: old
random_state: None
verbose: 0
model_path: 
model: IsolationForest(contamination=0.1, max_samples=1000, n_jobs=1)
algorithm_: isolationForest
label_parser: None
{'label_parser': None, 'algorithm_': 'isolationForest', 'contamination': 0.1, 'n_estimators': 100, 'max_samples': 1000, 'max_features': 1.0, 'bootstrap': False, 'n_jobs': 1, 'behaviour': 'old', 'random_state': None, 'verbose': 0, 'model_path': '', 'model': IsolationForest(contamination=0.1, max_samples=1000, n_jobs=1)}


In [6]:
modelIforest.fit(X_train,y_train)


Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 3

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 4

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 5

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 6

Training model at client.
Training model at client.
Training model at client.
Training 

In [7]:
scores_pred = modelIforest.decision_function(X_train)
print(scores_pred)
modelIforest.predict(X_train)

[0.4237302  0.37260277 0.38238652 ... 0.36813848 0.39052229 0.37683062]
Inspecting model's attributes:
contamination: 0.1
n_estimators: 100
max_samples: 1000
max_features: 1.0
bootstrap: False
n_jobs: 1
behaviour: old
random_state: None
verbose: 0
model_path: 
model: IsolationForest(contamination=0.1, max_samples=1000, n_jobs=1)
algorithm_: isolationForest
label_parser: None


In [8]:
print(modelIforest.model.d_scores_ == scores_pred)
labels_pred = modelIforest.model.labels_
labels_pred.shape

[ True  True  True ...  True  True  True]


(24000,)

In [9]:
modelIforest.evaluate(X_test,y_test)

Acc: 73.483% 

Precision: 0.280 

F1score: 0.174 

Recall: 0.127 

AUC_ROC: 0.517 



In [10]:
labels_true = y_test.ravel()
labels_true.shape

(6000,)

In [11]:
from RADAR.visualization_module import DataVisualization
visualizer = DataVisualization(data=X_test, plot_technique='plot_anomaly', dim_reduction_technique='PCA', y_true=labels_true, y_pred=labels_pred,subset_size_percent=0.2)
visualizer.fit()
visualizer.show()

##### AutoEncoder

In [4]:
from RADAR.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "autoencoder",
    "contamination":0.1,
    "label_parser": None,
     "epochs": 100,
     "input_dim": X_train.shape[1],
     "batch_size": 8,
     "neurons": [16,8, 16],
     "hidden_act": ['relu', 'relu', 'relu'],
     "n_clients":2,
     "n_rounds":10,
     }

modelAE = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelAE.get_params())


Federated Params:{'n_clients': 2, 'n_rounds': 10} 
 Model Params:{'algorithm_': 'autoencoder', 'contamination': 0.1, 'label_parser': None, 'epochs': 100, 'input_dim': 23, 'batch_size': 8, 'neurons': [16, 8, 16], 'hidden_act': ['relu', 'relu', 'relu']}
Inspecting model's attributes:
contamination: 0.1
input_dim: 23
neurons: [16, 8, 16]
model_path: 
w_size: None
n_pred: 1
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x720f24282d10>]
hidden_act: ['relu', 'relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 8
epochs: 100
optimizer: adam
preprocess: True
scaler: StandardScaler()
model: <Sequential name=sequential_1, built=True>
algorithm_: autoencoder
label_parser: None
{'label_parser': None, 'algorithm_': 'autoencoder', 'contamination': 0.1, 'input_dim': 23, 'neurons': [16, 8, 16], 'model_path': '', 'w_size': None, 'n_pred': 1, 'callbacks': [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x720f24282d10>], 'hidden_act': ['relu

/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-30 22:45:12.599491: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
modelAE.fit(X_train,y_train)


In [6]:
scores_pred = modelAE.decision_function(X_train)
print("Scores",scores_pred)
pred = modelAE.predict(X_train)

autoencoder, decision_function(): This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
Scores None
Inspecting model's attributes:
contamination: 0.1
input_dim: 23
neurons: [16, 8, 16]
model_path: 
w_size: None
n_pred: 1
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x720f24282d10>]
hidden_act: ['relu', 'relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 8
epochs: 100
optimizer: adam
preprocess: True
scaler: StandardScaler()
model: <Sequential name=sequential_1, built=True>
algorithm_: autoencoder
label_parser: None
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step


In [7]:
modelAE.model.d_scores_ ==scores_pred
labels_pred = modelAE.model.labels_
labels_pred.shape

(24000,)

In [8]:
labels_true = y_test.ravel()
labels_true.shape

(6000,)

In [10]:

modelAE.evaluate(X_test,y_test)

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step
Acc: 72.483% 

Precision: 0.230 

F1score: 0.143 

Recall: 0.104 

AUC_ROC: 0.503 



In [23]:
from RADAR.visualization_module import DataVisualization
visualizer = DataVisualization(data=X_test, plot_technique='plot_anomaly', dim_reduction_technique='PCA', y_true=labels_true, y_pred=labels_pred,subset_size_percent=0.2)
visualizer.fit()
visualizer.show()

##### Cluster

In [16]:
from RADAR.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "clusterAnomaly",
    "contamination":0.1,
    "label_parser": None,
    "n_clusters": 4,
    "n_clients":5,
    "n_rounds":10,
       
}

modelCluster = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelCluster.get_params())

Federated Params:{'n_clients': 5, 'n_rounds': 10} 
 Model Params:{'algorithm_': 'clusterAnomaly', 'contamination': 0.1, 'label_parser': None, 'n_clusters': 4}
Inspecting model's attributes:
contamination: 0.1
n_clusters: 4
seed: None
init_centroids: random
max_iter: 100
tol: 0.0001
verbose: True
n_init: 1
algorithm: lloyd
model_path: 
model: KMeans(init='random', max_iter=100, n_clusters=4, n_init=1)
algorithm_: clusterAnomaly
label_parser: None
{'label_parser': None, 'algorithm_': 'clusterAnomaly', 'contamination': 0.1, 'n_clusters': 4, 'seed': None, 'init_centroids': 'random', 'max_iter': 100, 'tol': 0.0001, 'verbose': True, 'n_init': 1, 'algorithm': 'lloyd', 'model_path': '', 'model': KMeans(init='random', max_iter=100, n_clusters=4, n_init=1)}


In [17]:
modelCluster.fit(X_train,y_train)



Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 3

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 4

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 5

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 6

Training model at client.
Training model at client.
Training model at client.
Training 

In [18]:
scores_pred = modelCluster.decision_function(X_train)
print(scores_pred)
modelCluster.predict(X_train)

[122256.1332972  101202.56907276  77515.42795219 ...  92612.4646725
  71078.74211753  47124.14167828]
Inspecting model's attributes:
contamination: 0.1
n_clusters: 4
seed: None
init_centroids: random
max_iter: 100
tol: 0.0001
verbose: True
n_init: 1
algorithm: lloyd
model_path: 
model: KMeans(init='random', max_iter=100, n_clusters=4, n_init=1)
algorithm_: clusterAnomaly
label_parser: None
cluster_centers_: [[ 3.07025630e+05  1.63229643e+00  1.60919080e+00  1.48379355e+00
   3.69099120e+01 -5.95297986e-01 -8.39176955e-01 -8.44245884e-01
  -8.58149115e-01 -8.68420946e-01 -8.82554425e-01  2.06583839e+04
   1.84501093e+04  1.79356330e+04  1.77207712e+04  1.72412485e+04
   1.71183094e+04  6.92287852e+03  7.44348367e+03  7.04973621e+03
   6.74588344e+03  6.80601702e+03  7.67674537e+03]
 [ 1.97325014e+05  1.60571696e+00  1.90183141e+00  1.53634841e+00
   3.57884681e+01  3.06680891e-01  2.89987420e-01  2.55706988e-01
   2.22204631e-01  1.76001307e-01  1.90396458e-01  1.34487221e+05
   1.31262

In [19]:
modelCluster.evaluate(X_test,y_test)

Acc: 70.783% 

Precision: 0.145 

F1score: 0.090 

Recall: 0.066 

AUC_ROC: 0.478 



##### PCA

In [20]:
from RADAR.federated_data.algorithms import flexanomalies

kwargs = {
    "algorithm_": "pcaAnomaly",
    "contamination":0.1,
    "label_parser": None,
    "preprocess":False,  
    "n_components": 4,
    "n_clients":5,
    "n_rounds":10,      
}

modelPca = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelPca.get_params())

Federated Params:{'n_clients': 5, 'n_rounds': 10} 
 Model Params:{'algorithm_': 'pcaAnomaly', 'contamination': 0.1, 'label_parser': None, 'preprocess': False, 'n_components': 4}
Inspecting model's attributes:
contamination: 0.1
n_components: 4
n_selected_components: None
copy: True
whiten: False
svd_solver: auto
tol: 0.0
iterated_power: auto
random_state: None
weighted: (True,)
preprocess: False
model_path: 
scaler: StandardScaler()
model: PCA(n_components=4)
algorithm_: pcaAnomaly
label_parser: None
{'label_parser': None, 'algorithm_': 'pcaAnomaly', 'contamination': 0.1, 'n_components': 4, 'n_selected_components': None, 'copy': True, 'whiten': False, 'svd_solver': 'auto', 'tol': 0.0, 'iterated_power': 'auto', 'random_state': None, 'weighted': (True,), 'preprocess': False, 'model_path': '', 'scaler': StandardScaler(), 'model': PCA(n_components=4)}


In [21]:
modelPca.fit(X_train,y_train)


Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 3

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 4

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 5

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 6

Training model at client.
Training model at client.
Training model at client.
Training 

In [22]:
scores_pred = modelPca.decision_function(X_train)
print(scores_pred)
modelPca.predict(X_train)

pcaAnomaly, decision_function(): 'PCA_Anomaly' object has no attribute 'selected_components_'
None
Inspecting model's attributes:
contamination: 0.1
n_components: 4
n_selected_components: None
copy: True
whiten: False
svd_solver: auto
tol: 0.0
iterated_power: auto
random_state: None
weighted: (True,)
preprocess: False
model_path: 
scaler: StandardScaler()
model: PCA(n_components=4)
algorithm_: pcaAnomaly
label_parser: None


In [23]:
modelPca.evaluate(X_test,y_test)

Acc: 71.317% 

Precision: 0.172 

F1score: 0.107 

Recall: 0.078 

AUC_ROC: 0.486 



##### Test: new model and new funtions primitives

In [12]:
from RADAR.federated_data.algorithms import flexanomalies
class MyAnomalyModel:
    def __init__(self, param1, param2):
        self.param1 = param1
        self.param2 = param2

flexanomalies.FlexAnomalyDetection.register_algorithm("myAnomalyModel", MyAnomalyModel)

# Ahora puedes instanciar usando el nuevo algoritmo
detector = flexanomalies.FlexAnomalyDetection(algorithm_="myAnomalyModel",param1=10, param2=20)
detector.register_federated_functions("customAnomaly", {
    "build_model": lambda: print("Building Custom Model"),
    "copy": lambda: print("Copying Custom Model"),
    "train": lambda: print("Training Custom Model"),
    "collect": lambda: print("Collecting Custom Model Weights"),
    "aggregate": lambda: print("Aggregating Custom Model Weights"),
    "set_weights": lambda: print("Setting Aggregated Weights"),
})

Federated Params:{} 
 Model Params:{'algorithm_': 'myAnomalyModel', 'param1': 10, 'param2': 20}


In [13]:
#Example using a federated external data set but using flex dataset format 
from flexanomalies.utils.load_data import federate_data
detector.flex_data = federate_data(5, X_train, y_train)